In [2]:
# Loading the dataset
# Using pandas
import pandas as pd
# Reading dataset
dataset = pd.read_table('ratings.dat', header = None, sep = '::', engine = 'python', usecols = [0,1,2], names = ('UserID','MovieID', 'Ratings'))

dataset.head()

# Are there missing values?
dataset.isna().sum()


# Keep in mind that there are “gaps” in numbering of users and items. (Dictionaries? Renumber everything? …)
# What is meant with this??

UserID     0
MovieID    0
Ratings    0
dtype: int64

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold 
import math
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import mean_squared_error



In [3]:
import matplotlib.pyplot as plt
plt.hist(dataset['Ratings'])
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [5]:
# How many unique Users and Movies there are

num_userID  = len(dataset.UserID.unique())
num_movieID = len(dataset.MovieID.unique())
#(num_userID,num_movieID)




hello
0.0


In [6]:
# Taks 1.1 Naive Approaches
import time

# Takes a long time to run but I think it works sort of correct, have also have to add everything for test 
start = time.time()
n_splits = 5
KF = KFold(n_splits=n_splits, random_state=123, shuffle=True)
GlobalAvergage_RMSE, UserAverage_RMSE, MovieAverage_RMSE, LinReg_RMSE, LinRegInter_RMSE = list(),list(),list(),list(),list()
GlobalAvergage_MAE, UserAverage_MAE, MovieAverage_MAE, LinReg_MAE, LinRegInter_MAE = list(),list(),list(),list(),list()
for train, test in KF.split(dataset):
    #Define X and y in train and test
    X_train, X_test = dataset.iloc[train, 0:2], dataset.iloc[test, 0:2]
    y_train, y_test = dataset.iloc[train, -1], dataset.iloc[test, -1]
    
    # Models
    GlobalAvergage = y_train.mean()
    UserAverage = dataset.iloc[train]
    MovieAverage = dataset.iloc[train]
    
    # Create dataframe for Linear Regression
    UserAverage = UserAverage['Ratings'].groupby(UserAverage['UserID'], group_keys=False).transform('mean') 
    MovieAverage = MovieAverage['Ratings'].groupby(MovieAverage['MovieID'], group_keys=False).transform('mean')
    X_lr = pd.DataFrame(columns = ['UserAverage', 'MovieAverage'])
    X_lr['UserAverage'] = UserAverage
    X_lr['MovieAverage'] = MovieAverage   
    modelLinReg = LR(random_state=123, fit_intercept=False, max_iter= 10000).fit(X_lr,y_train)
    modelLinRegInter = LR(random_state=123, fit_intercept=True, max_iter= 10000).fit(X_lr,y_train)
    
    
    # Predictions 
    pred_GlobalAvergage = [GlobalAvergage] * len(X_train) 
    pred_UserAverage = UserAverage
    pred_MovieAverage = MovieAverage
    pred_LinReg = modelLinReg.predict(X_lr)
    pred_LinRegInter = modelLinRegInter.predict(X_lr)

    #RMSE's 
    RMSE_GlobalAvergage = math.sqrt(mean_squared_error(pred_GlobalAvergage,y_train))
    RMSE_UserAverage = math.sqrt(mean_squared_error(pred_UserAverage,y_train))
    RMSE_MovieAverage = math.sqrt(mean_squared_error(pred_MovieAverage,y_train))
    RMSE_LinReg = math.sqrt(mean_squared_error(pred_LinReg,y_train))
    RMSE_LinRegInter = math.sqrt(mean_squared_error(pred_LinRegInter,y_train))
    
    GlobalAvergage_RMSE.append(RMSE_GlobalAvergage)
    UserAverage_RMSE.append(RMSE_UserAverage)
    MovieAverage_RMSE.append(RMSE_MovieAverage)
    LinReg_RMSE.append(RMSE_LinReg)
    LinRegInter_RMSE.append(RMSE_LinRegInter)
    
    # MAE's 
    MAE_GlobalAvergage = mae(pred_GlobalAvergage, y_train)
    MAE_UserAverage = mae(pred_UserAverage, y_train)
    MAE_MovieAverage = mae(pred_MovieAverage, y_train)
    MAE_LinReg = mae(pred_LinReg, y_train)
    MAE_LinRegInter = mae(pred_LinRegInter, y_train)
    
    GlobalAvergage_MAE.append(MAE_GlobalAvergage)
    UserAverage_MAE.append(MAE_UserAverage)
    MovieAverage_MAE.append(MAE_MovieAverage)
    LinReg_MAE.append(MAE_LinReg)
    LinRegInter_MAE.append(MAE_LinRegInter)
    
end = time.time()
print(end - start)

135.6486690044403


In [6]:
Average_RMSE_GlobalAvergage = np.mean(GlobalAvergage_RMSE)
Average_MAE_GlobalAvergage =  np.mean(GlobalAvergage_MAE)

Average_RMSE_UserAverage = np.mean(UserAverage_RMSE)
Average_MAE_UserAverage =  np.mean(UserAverage_MAE)

Average_RMSE_MovieAverage = np.mean(MovieAverage_RMSE)
Average_MAE_MovieAverage = np.mean(MovieAverage_MAE)

Average_RMSE_LinReg = np.mean(LinReg_RMSE)
Average_MAE_LinReg = np.mean(LinReg_MAE)

Average_RMSE_LinRegInter = np.mean(LinRegInter_RMSE)
Average_MAE_LinRegInter = np.mean(LinRegInter_MAE)

[0.6963733820564957, 0.696703313183373, 0.6956247883254371, 0.6961271834504548, 0.6953664730406615]


## Task 1.2


In [5]:
dataset_task2 = dataset.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
)

In [5]:
def Preprocessing(dataset):
    dataset1 = dataset.sub(dataset.mean(axis = 1), axis=0)
    dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1) #not needed? normalizing among movie axis
    return dataset1
Preprocessing(dataset_task2)

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.811321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,-1.146465,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,-1.302928,NaN,-0.302928,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def Initialization(dataset):
    d = 5
    n = dataset.shape[0]
    m = dataset.shape[1]
    a = dataset.stack().mean()
    U = np.empty([n,d])
    V = np.empty([d,m])
    U.fill(math.sqrt(a/d))
    V.fill(math.sqrt(a/d))
    return U,Vb



(array([[0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937],
        [0.8442937, 0.8442937, 0.8442937, 0.8442937, 0.8442937]]),
 array([[0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937],
        [0.8442937, 0.8442937, 0.8442937, ..., 0.8442937, 0.8442937,
         0.8442937]]))

In [5]:
dataset1 = dataset.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1) #not needed? normalizing among movie axis

d = 5
n = dataset_end.shape[0]
m = dataset_end.shape[1]
a = dataset_end.stack().mean()
U = np.empty([n,d])
V = np.random.uniform(low = -2, high = 2, size = (d, m)) # random numbers to increase chance of reachine global minimum
U = np.random.randn(n, d)

print(V.shape, U.shape)

(5, 3) (1000209, 5)


In [11]:
dataset_task2 = dataset.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
)

dataset1 = dataset_task2.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1)


#print(dataset.sort_values(['UserID', 'MovieID']),df_test.sort_values(by=['UserID', 'MovieID']).dropna())
#pd.melt(dataset_task2, id_vars='')

In [7]:
#initialize vars
dataset1 = dataset_task2.sub(dataset_task2.mean(axis = 1), axis=0)
dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1)

M = dataset_end.to_numpy()
d = 2
n = dataset_end.shape[0]
m = dataset_end.shape[1]
a = dataset_end.stack().mean()
U = np.empty([n,d])
V = np.random.uniform(low = -2, high = 2, size = (d, m)) # random numbers to increase chance of reachine global minimum
U = np.random.randn(n, d)
uv = np.matmul(U,V)


#make train and test sets



for 
#U
for r in range(U.shape[0]):
    numerator = 0
    denominator = 0

    for s in range(d):
        U_rk = U[r,:]
        U_rk = np.delete(U_rk, s, 0)
        V_kj = np.delete(V, s, 0)
        V_sj = V[s,:]
        P = np.matmul(U_rk, V_kj)
        m_rj = M[r,:]
        numerator = V_sj * (m_rj - P)
        numerator = np.nansum(numerator * (np.isfinite(m_rj)))
        denominator = V_sj ** 2
        denominator = np.nansum(denominator * (np.isfinite(m_rj)))
        new_var = numerator/denominator
        U[r,s] = new_var

for r in range(d):
    for s in range(V.shape[1]):
        V_ks = V[:,s]
        V_ks = np.delete(V_ks, r, 0)
        U_ik = np.delete(U, r, 1)
        U_ir = U[:,r]
        P = np.matmul(U_ik, V_ks)
        m_is = M[:,s]
        numerator = U_ir * (m_is - P)
        numerator = np.nansum(numerator * (np.isfinite(m_is)))
        denominator = U_ir ** 2
        denominator = np.nansum(denominator * (np.isfinite(m_is)))
        newvar = numerator/denominator
        V[r,s] = newvar

#rewrite this part
uv = np.matmul(U,V)
dif = uv - M
dif_sqr = dif ** 2
dif_sqr_0s = np.nan_to_num(dif_sqr)
dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
sumz = dif_sqr_total.sum()
non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
RME = sumz/ non_0_count_sqr
rme_list=[RME]
print('RMSE=',RME)

RMSE= 3.559348380929595
RMSE= 0.8287853060409066


In [90]:
import time
start = time.time()
n_splits = 5
KF = KFold(n_splits=5, random_state=123, shuffle=True)
for Train, Test in KF.split(dataset):
    dataset_2_Train, dataset_2_Test = dataset.loc[Train], dataset.loc[Test]
    dataset_split = dataset_2_Train.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
    )
    M = dataset_split.to_numpy()
    d = 2
    n = dataset_split.shape[0]
    m = dataset_split.shape[1]
    a = dataset_split.stack().mean()
    U = np.empty([n,d])
    V = np.random.randn(d, m) # random numbers to increase chance of reachine global minimum
    U = np.random.randn(n, d)
    uv = np.matmul(U,V)
    
    rme_list = list()
    RME = 1000
    halt = True
    for i in range(10):
        RME_old = RME
        for r in range(U.shape[0]):
            numerator = 0
            denominator = 0

            for s in range(d):
                U_rk = U[r,:]
                U_rk = np.delete(U_rk, s, 0)
                V_kj = np.delete(V, s, 0)
                V_sj = V[s,:]
                P = np.matmul(U_rk, V_kj)
                m_rj = M[r,:]
                numerator = V_sj * (m_rj - P)
                numerator = np.nansum(numerator * (np.isfinite(m_rj)))
                denominator = V_sj ** 2
                denominator = np.nansum(denominator * (np.isfinite(m_rj)))
                new_var = numerator/denominator
                U[r,s] = new_var

        for r in range(d):
            for s in range(V.shape[1]):
                V_ks = V[:,s]
                V_ks = np.delete(V_ks, r, 0)
                U_ik = np.delete(U, r, 1)
                U_ir = U[:,r]
                P = np.matmul(U_ik, V_ks)
                m_is = M[:,s]
                numerator = U_ir * (m_is - P)
                numerator = np.nansum(numerator * (np.isfinite(m_is)))
                denominator = U_ir ** 2
                denominator = np.nansum(denominator * (np.isfinite(m_is)))
                newvar = numerator/denominator
                V[r,s] = newvar
        
        uv = np.matmul(U,V)
        dif = uv - M
        dif_sqr = dif ** 2
        dif_sqr_0s = np.nan_to_num(dif_sqr)
        dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
        sumz = dif_sqr_total.sum()
        non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
        RME = sumz/ non_0_count_sqr
        
        halt = np.abs(RME - RME_old) > 0.001
        print('RMSE=',RME)
    rme_list.append(RME)
print('Avg=', np.mean(rme_list))
ende = time.time()
print((ende - start)/60)

RMSE= 12.69381352892171
RMSE= 1.2894802682149429
RMSE= 0.8247116144450103
RMSE= 0.8060971574802386
RMSE= 0.799661589975494
RMSE= 0.7957755207154923
RMSE= 0.7928265969755862
RMSE= 0.7903483330717564
RMSE= 0.7881648137382953
RMSE= 0.7861993025586739
RMSE= 8.850928780897243
RMSE= 0.9461802287716399
RMSE= 0.8127523343093269
RMSE= 0.8084304361381401
RMSE= 0.8078495701634277
RMSE= 0.8076000763685355
RMSE= 0.8074154406842813
RMSE= 0.8072443400294144
RMSE= 0.8070733464726999
RMSE= 0.8068996970638569
RMSE= 12.728380857453452
RMSE= 1.8173875373857846
RMSE= 0.861810603441364
RMSE= 0.817539250587369
RMSE= 0.8084694966795417
RMSE= 0.8047586707945864
RMSE= 0.8023235034741846
RMSE= 0.8002399015439948
RMSE= 0.7985996111272046
RMSE= 0.7973864733740795
RMSE= 9.590417742507901
RMSE= 0.9649460706853727
RMSE= 0.8024195938757068
RMSE= 0.7887001104033861
RMSE= 0.7800360907878283
RMSE= 0.7726397652526953
RMSE= 0.766404270091172
RMSE= 0.7613067644187305
RMSE= 0.7572065090125643
RMSE= 0.7539452600405159
RMSE= 8

In [5]:
#test + optimization cell
import time
start = time.time()
#initialize params
n_iter = 0
rme_list = list()
n_splits = 5
Threshold = 0.001
KF = KFold(n_splits=5, random_state=123, shuffle=True)
#Kfold loop (5 datasets)
for Train, Test in KF.split(dataset):
    dataset_2_Train, dataset_2_Test = dataset.loc[Train], dataset.loc[Test]
    dataset_split = dataset_2_Train.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
    )
    dataset1 = dataset_split.sub(dataset_split.mean(axis = 1), axis=0)
    dataset_end = dataset1.sub(dataset1.mean(axis = 0), axis = 1)
    dataset_split = dataset_end
    M = dataset_split.to_numpy()
    d = 2
    n = dataset_split.shape[0]
    m = dataset_split.shape[1]
    a = dataset_split.stack().mean()
    U = np.empty([n,d])
    V = np.random.randn(d, m) # random numbers to increase chance of reachine global minimum
    U = np.random.randn(n, d)
    uv = np.matmul(U,V)
    
    RME = 1000
    halt = True
    n_iter += 1
    #iterate as long as change of RME is bigger than threshold
    while halt:
        RME_old = RME
        #update U matrix
        for r in range(U.shape[0]):
            numerator = 0
            denominator = 0

            for s in range(d):
                U_rk = U[r,:]
                U_rk = np.delete(U_rk, s, 0)
                V_kj = np.delete(V, s, 0)
                V_sj = V[s,:]
                P = np.matmul(U_rk, V_kj)
                m_rj = M[r,:]
                numerator = np.multiply(V_sj,(np.subtract(m_rj, P)))
                numerator = np.nansum(np.multiply(numerator, (np.isfinite(m_rj))))
                denominator = np.square(V_sj)
                denominator = np.nansum(np.multiply(denominator, (np.isfinite(m_rj))))
                new_var = np.divide(numerator, denominator)
                U[r,s] = new_var
        #update V matrix
        for r in range(d):
            for s in range(V.shape[1]):
                V_ks = V[:,s]
                V_ks = np.delete(V_ks, r, 0)
                U_ik = np.delete(U, r, 1)
                U_ir = U[:,r]
                P = np.matmul(U_ik, V_ks)
                m_is = M[:,s]
                numerator = np.multiply(U_ir, (np.subtract(m_is, P)))
                numerator = np.nansum(np.multiply(numerator, (np.isfinite(m_is))))
                denominator = np.square(U_ir)
                denominator = np.nansum(np.multiply(denominator, (np.isfinite(m_is))))
                newvar = np.divide(numerator, denominator)
                V[r,s] = newvar
        #matrix multiplication and comparison to original matrix M + calc MSE
        uv = np.matmul(U,V)
        dif_squared_0 = np.nan_to_num(np.square(np.subtract(uv, M)))
        dif_squared_total_sum = np.sum(dif_squared_0, axis = 0).sum()
        N_non_0 = np.count_nonzero(dif_squared_0)
        RME = np.divide(dif_squared_total_sum, N_non_0)
        RMSE = np.sqrt(RME)
        halt = np.abs(RMSE - RME_old) > Threshold
    print(f'fold {n_iter}; RMSE = {RMSE}')
    rme_list.append(RME)
print('Avg=', np.mean(rme_list))
rme_list_mean = np.mean(rme_list)
print(f'Average RMSE over {n_iter} folds = {rme_list_mean}')
ende = time.time()
print('Total runtime = ', (ende - start))

fold 1; RMSE = 0.890533737922338
fold 2; RMSE = 0.8975831949207081
fold 3; RMSE = 0.8985919112431703
fold 4; RMSE = 0.8915676354426462
fold 5; RMSE = 0.8949593179375439
Avg= 0.8004036764931353
Average RMSE over 5 folds = 0.8004036764931353
Total runtime =  92.63752484321594


In [6]:
start = time.time_ns()
dif = uv - M
dif_sqr = dif ** 2
dif_sqr_0s = np.nan_to_num(dif_sqr)
dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
sumz = dif_sqr_total.sum()
non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
RME = sumz/ non_0_count_sqr
end = time.time_ns()
print(RME)

0.7593419312438067


In [69]:
start = time.time_ns()
dif_sqr_total= np.nansum(np.square(np.subtract(uv, M)))
non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
RME = np.divide(dif_sqr_total, non_0_count_sqr)
end = time.time_ns()
print(RME)

16.032950606445226
